# Modelling of France Accidents

**Cohort:** mar23_accidents

**Author:** Tobias Schulze

**Date:** 30 Oktober 2023



## Loading of required libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import re
import pickle
import warnings
import time
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from datetime import datetime, timedelta
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

## Importing the data

In [2]:
df = pd.read_csv('./data/231030_clean_table_for_analysis.csv', low_memory = False, index_col = 0)

KeyboardInterrupt: 

## Data description
### Data

In [ ]:
df.head()

### Data types

In [ ]:
df.info()

### Missing data

In [ ]:
df.isna().sum()

This data has no missing values, accept the `holiday` variable which contains the name of the holiday or NaN.

### Drop variables
The dataset contains still some variables that represent information represented by other variables or are a finer granulation. The latter might be added later, if the primary variable is relevant.

- `holiday`: classifies the holiday, but will be used only, if `is_holiday` is relevant

In [ ]:
columns_drop = ['holiday']

df.drop(columns = columns_drop, inplace = True)

In [ ]:
df.info()

## Data preparation
### Transform data types
#### Date and time variables
The date and time variables are maybe important as grouping variables or as contrains for time dependent severity of accidents.

_Problem_:

Due to the export to `csv`, the `datatime` data time is lost and thus, it is required to assign it again.

**Steps:**
1. Transform `date` and `hrmn` to `datetime` type.nais` to integer and `date` to `date`.

In [ ]:
# Fixing the hrmn issue:
# Transform `date`
df = df.assign(date = pd.to_datetime(df['date']))
df = df.assign(hrmn = pd.to_datetime(df['hrmn']))

In [ ]:
df['date'] = df.apply(lambda x: int(x['date'].timestamp()), axis = 1)
df['hrmn'] = df.apply(lambda x: int(x['hrmn'].timestamp()), axis = 1)

In [ ]:
df.info()

## Data analysis
### Correlation matrix
To get a first glimpse on the possible contrains in the data, a correlation matrix is plotted.

The data variables are dropped to remove redundany with `timestamp`.

In [ ]:
df.info()

In [ ]:
# Drop date columns for the correlation matrix
columns_drop = ['an', 'mois', 'jour', 'datetime', 'timestamp'] #, 'date']
df_select = df.drop(columns_drop, axis = 1)

In [ ]:
# Scale the data
scaler = MinMaxScaler()
df_numeric_scaled = pd.DataFrame(scaler.fit_transform(df_select), columns=df_select.columns)

In [ ]:
# Create a correlation matrix
correlation_matrix = df_numeric_scaled.corr()
plt.figure(figsize=(10, 8))
# Generate the heatmap
sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm')

# Add a title
plt.title('Correlation Heatmap')

# Display the plot
plt.show();

The correlation matrix shows some intervariable correlations, but no real importance of variables to predict the target `fatal`. 

### Preparation of the training and test datasets
#### Split data
The dataset is split by year slides. The years 2005 to 2019 are selected for the train dataset and the years 2020 and 2021 for the test dataset.


In [ ]:
# split the dataset to train and test data
X_train = df.loc[df['an'] < 2020].drop('grav', axis = 1)
X_test = df.loc[df['an'] > 2019].drop('grav', axis = 1)

y_train = df['grav'].loc[df['an'] < 2020]
y_test = df['grav'].loc[df['an'] > 2019]


from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)


#### Data scaling
Decision trees are not sensitive to different scales and thus scaling is not required.

### Modelling the data using a XGBoost Classification Model with Tree-based Parzen Estimators optimisation
The Tree-based Parzen Estimator (TPE) [1] optimisation combines a Bayesian Sequential Model Based Optimisation (SMBO) and a random search on the hyperparemeter grid[2]. While the random search is a static approach, the SMBO optimises the model using prior runs to determine future points of exploration.

The package `hyperopt` [4] is used for optimisation. The hyperparameter optimisation code is written based on ideas in [5].

#### The tuning space
`hyperopt` uses a specific format of parameters expressions [7]. Hints on the parameter ranges are given in [1,5,7].

The `booster` in this first model is `gbtree`. The evaluation metric is `logloss` or `auc`[8].

Another classification method is Dropout with Multiple Additive Regression Trees (`dart`)[9], which is not used here.

The metric used is `mlogloss` (as a special case of `logloss` for multiclass modelling). The reason is that `logloss` is an absolute measure of the quality of the classification, while `auc` is a simple ranking function [10]. `logloss` is discussed as better performing with imbalanced datasets [11].


In [ ]:
# Create the tuning space
booster = 'gbtree'
eval_metric = 'mlogloss'

tuning_space={
    'eta': hp.uniform('eta', 0, 1),
    'max_depth': hp.quniform('max_depth', 3, 10, 1),
    'gamma': hp.uniform ('gamma', 0, 10),
    'reg_alpha' : hp.uniform('reg_alpha', 0, 100),
    'reg_lambda' : hp.uniform('reg_lambda', 0, 10),
    'colsample_bytree' : hp.uniform('colsample_bytree', 0, 1),
    'colsample_bylevel' : hp.uniform('colsample_bylevel', 0, 1),
    'colsample_bynode' : hp.uniform('colsample_bynode', 0, 1),
    'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
    'n_estimators': hp.quniform('n_estimators', 10, 1000, 1), # default is 100, so we try to check in the one magnitude space
    'seed': 123,
    'early_stopping_rounds': 10, # stop tuning early to avoid overfitting
    'objective': 'binary:logistic',
    'subsample': hp.uniform('subsample', 0, 1)
    }

tuning_space.update({'scale_pos_weight': hp.uniform('scale_pos_weight', 0, 2)}) # helps in auc
tuning_space.update({'max_delta_step': hp.uniform('max_delta_step', 0, 2)}) # helps in auc
tuning_space.update({'eval_metric': eval_metric})
tuning_space.update({'booster': booster})


#### The tuning function
The following function integrates all steps required for the hyperoptimization tuning. The results are stored in the model and the tuning score is printed per step.

In [ ]:
# Function for the classification
def hp_xgbclass(space, X_train = X_train, y_train = y_train, X_test = X_test, y_test = y_test):
    """ Function for Bayesian hyperparameter optimisation of XGboost classification model.
    Accepts a parameter `space` and the training and testing data set.
        :param tuning_space: a catalog containing the parameters used for hyperparameter tuning, ranges given in hp objects
        :param X_train: accepts the train data
        :param X_test: accepts the test data
        :param X_eval: accepts the eval data used for optimisation
        :param y_train: accepts the train target
        :param y_test: accepts the test target
        :param y_eval: accepts the eval data used for optimisation
    """

    if (space['objective'] == 'binary:logistic') and (space['booster'] == 'gbtree'):
        clf=xgb.XGBClassifier(
            eta = space['eta'],
            eval_metric = space['eval_metric'],
            booster = space['booster'],
            n_estimators = np.int64(space['n_estimators']),
            max_depth = np.int64(space['max_depth']),
            gamma = space['gamma'],
            reg_alpha = space['reg_alpha'],
            min_child_weight=np.int64(space['min_child_weight']),
            colsample_bytree=space['colsample_bytree'],
            colsample_bylevel=space['colsample_bylevel'],
            colsample_bynode=space['colsample_bynode'],
            early_stopping_rounds = space['early_stopping_rounds'],
            objective = space['objective'],
            # scale_pos_weight = space['scale_pos_weight'], # not used in this model
            max_delta_step = space['max_delta_step'],
            subsample = space['subsample']
    
        )
          
    clf.fit(X_train,
            y_train,
            eval_set = [(X_test, y_test)],
            verbose=False)
        
    pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, pred > 0.5)
    print ("SCORE:", accuracy)
    return {'loss': -accuracy, 'status': STATUS_OK}

#### The tuning
In this section, the hyperparameter tuning is performed. The results are stored in the trial db and the hyperparameter file for later re-use. In case, the downstream analysis is changed, the model could be loaded in this section.

In [ ]:
# Running the crossvalidation or load stored parameters

# lets check the run time
start_time = time.monotonic()

# Suppress warnings because of deprecated functions or parameters, we cannot influence
warnings.filterwarnings('ignore')

# Rerun (True) or get stored parameters (False)
cv_rerun = True

# define the stored files
out_file_trial = './data/XGboost_model/231030_XGboost_gbtree_mlogloss_grav_non_encoded'
out_file_params = './data/XGboost_model/231030_XGboost_gbtree_mlogloss_hyperparameters_grav_non_encoded'


# rerun the crossvalidation
if (cv_rerun == True):
    trials = Trials()
    best_hyperparams = fmin(fn = hp_xgbclass,
                            space = tuning_space,
                            algo = tpe.suggest,
                            max_evals = 100,
                            trials = trials,
                            verbose=0
                           )
    # save the best parameters and the trials database
    with open(out_file_trial, 'wb') as f:
        pickle.dump(trials, f)

    with open(out_file_params, 'wb') as f:
        pickle.dump(best_hyperparams, f)

# get the stored hyperparameters
else:
    with open('./data/XGboost_model/231030_XGboost_gbtree_mlogloss_hyperparameters_grav_raw', 'rb') as f:
        best_hyperparams = pickle.load(f)

# Track the end of the tuning
end_time = time.monotonic()
print(timedelta(seconds=end_time - start_time))

In [ ]:
print("The best hyperparameters are : ","\n")
print(best_hyperparams)

In [ ]:
xgb_1 = xgb.XGBClassifier(
    booster = tuning_space['booster'],
    eta = best_hyperparams['eta'],
    gamma = best_hyperparams['gamma'],
    max_depth = np.int64(best_hyperparams['max_depth']),
    min_child_weight = np.int64(best_hyperparams['min_child_weight']),
    reg_alpha = best_hyperparams['reg_alpha'],
    reg_lambda = best_hyperparams['reg_lambda'],
    eval_metric = tuning_space['eval_metric'],
    objective = tuning_space['objective'],
    seed = tuning_space['seed'],
    early_stopping_rounds = tuning_space['early_stopping_rounds'],
    n_estimators = np.int64(best_hyperparams['n_estimators']),
    subsample = best_hyperparams['subsample'],
    max_delta_step = best_hyperparams['max_delta_step'],
    colsample_bylevel = best_hyperparams['colsample_bylevel'],
    colsample_bynode = best_hyperparams['colsample_bynode'],
    colsample_bytree = best_hyperparams['colsample_bytree']
        
    )

xgb_1 = xgb_1.fit(X_train,
          y_train,
          eval_set = [(X_test, y_test)], 
          verbose = False)


In [ ]:
importance_types = ['total_gain', 'gain', 'weight', 'cover', 'total_cover']

for f in importance_types:
    xgb.plot_importance(xgb_1, max_num_features=50, importance_type=f, title=f)
    plt.figure(figsize = (16, 12))
    plt.show();

In [ ]:
y_pred = xgb_1.predict(X_test)
y_pred = [round(value) for value in y_pred]

print(classification_report(y_pred, y_test))

print(pd.crosstab(y_pred, y_test))

In [ ]:
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize = (5, 4))
sns.set(font_scale=1)
sns.heatmap(cm, annot=True);

In [ ]:
from xgboost import plot_tree
import graphviz
plot_tree(xgb_1)
plt.show()


## References
[1] [Bergstra, J., Yamins, D., Cox, D. D. (2013) Making a Science of Model Search: Hyperparameter Optimization in Hundreds of Dimensions for Vision Architectures. TProc. of the 30th International Conference on Machine Learning (ICML 2013), June 2013, pp. I-115 to I-23.](https://proceedings.neurips.cc/paper_files/paper/2011/file/86e8f7ab32cfd12577bc2619bc635690-Paper.pdf)

[2] https://towardsdatascience.com/hyperopt-demystified-3e14006eb6fa

[3] https://xgboost.readthedocs.io/en/stable/tutorials/categorical.html

[4] https://github.com/hyperopt/hyperopt

[5] https://www.kaggle.com/code/prashant111/a-guide-on-xgboost-hyperparameters-tuning

[6] https://github.com/hyperopt/hyperopt/wiki/FMin

[7] https://xgboost.readthedocs.io/en/stable/tutorials/param_tuning.html

[8] https://www.datamachines.io/blog/auc-vs-log-loss

[9] [Rashmi, K.V., Gilad-Bachrach (2015) DART: Dropouts meet Multiple Additive Regression Trees. arXiv:1505.01866v](https://doi.org/10.48550/arXiv.1505.01866)

[10] https://datamachines.com/blog/auc-vs-log-loss

[11] https://stats.stackexchange.com/questions/322408/logloss-vs-gini-auc